In [1]:
import pyspark
import pandas as pd
#import datefinder
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import re   
import seaborn as sns
import statsmodels.api as sm
#from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
from fbprophet import Prophet
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import requests

Importing plotly failed. Interactive plots will not work.


In [2]:
sc = pyspark.SparkContext(appName="Spark RDD")

CLEANING THE DATA

In [3]:
#Remove tuple with len !=14
def RemoveStrangeTupleLen(tup):
    if len(tup)==14:
        return tup
    
#CLEANING MAC_ADDRESS_SRC FIELD
def RemoveQuoteSrc(tup):
    if tup[0][0]=='"':
        tup[0]=tup[0][1:]
    return tup

def RemoveWeirdAddress(tup):
    if len(tup[0])>17:
        tup[0]=None
    return tup

#CLEANING CAT FIELD
def CATToInt(tup): 
    tup[1] = int(tup[1])
    return tup

#CLEANING TID FIELD
def replaceNullValue_TID(tup):
    if tup[2] != '' and tup[2] != None and tup[2] != 'NaN': 
        return tup
    else: 
        tup[2] = ''
        return tup
    
#CLEANING TS FIELD
def TSToFloat(tup): 
    tup[3] = float(round(float(tup[3])))
    return tup

#CLEANING DST FIELD
#None

#CLEANING SAC FIELD
def replaceNullValue_SAC(tup):
    if tup[5] != '' and tup[5] != None and tup[5] != 'NaN': 
        tup[5] = float(tup[5])
        return tup
    else: 
        tup[5] = None
        return tup

#CLEANING SIC FIELD
def replaceNullValue_SIC(tup):
    if tup[6] != '' and tup[6] != None and tup[6] != 'NaN': 
        tup[6] = float(tup[6])
        return tup
    else: 
        tup[6] = None
        return tup
    
#CLEANING ToD FIELD
def replaceNullValue_ToD(tup):
    if tup[7] != '' and tup[7] != None and tup[7] != 'NaN': 
        tup[7] = float(tup[7])
        return tup
    else: 
        tup[7] = None
        return tup

#CLEANING TN FIELD
def replaceNullValue_TN(tup):
    if tup[8] != '' and tup[8] != None and tup[8] != 'NaN': 
        tup[8] = float(tup[8])
        return tup
    else: 
        tup[8] = None
        return tup

#CLEANING THETA FIELD
def replaceNullValue_THETA(tup):
    if tup[9] != '' and tup[9] != None and tup[9] != 'NaN': 
        tup[9] = float(tup[9])
        return tup
    else: 
        tup[9] = None
        return tup

#CLEANING RHO FIELD
def replaceNullValue_RHO(tup):
    if tup[10] != '' and tup[10] != None and tup[10] != 'NaN': 
        tup[10] = float(tup[10])
        return tup
    else: 
        tup[10] = None
        return tup
    
#CLEANING FL FIELD 
def replaceNullValue_FL(tup):
    if tup[11] != '' and tup[11] != None and tup[11] != 'NaN': 
        tup[11] = float(tup[11])
        return tup
    else: 
        tup[11] = None
        return tup
    
#CLEANING CGS FIELD
def replaceNullValue_CGS(tup):
    if tup[12] != '' and tup[12] != None and tup[12] != 'NaN': 
        tup[12] = float(tup[12])
        return tup
    else: 
        tup[12] = None
        return tup
    
#CLEANING CHdg FIELD
    
def replaceNullValue_CHdg(tup):
    if tup[13] != '' and tup[13] != None and tup[13] != 'NaN': 
        tup[13] = float(tup[13])
        return tup
    else: 
        tup[13] = None
        return tup
    
def RemoveQuoteCHdg(tup):
    if tup[13][-1]=='"':
        tup[13]=tup[13][:-1]
    return tup


def cleaning(tup):
    tup = RemoveQuoteSrc(tup)
    tup = RemoveWeirdAddress(tup)
    tup = CATToInt(tup)
    tup = replaceNullValue_TID(tup)
    tup = TSToFloat(tup)
    tup = replaceNullValue_SAC(tup)
    tup = replaceNullValue_SIC(tup)
    tup = replaceNullValue_ToD(tup)
    tup = replaceNullValue_TN(tup) 
    tup = replaceNullValue_THETA(tup)
    tup = replaceNullValue_RHO(tup)
    tup = replaceNullValue_FL(tup)
    tup = replaceNullValue_CGS(tup)
    tup = replaceNullValue_CHdg(RemoveQuoteCHdg(tup))
    return tup
    
def main_clean(rdd):
    
    #header = rdd.first()
    #rdd = rdd.filter(lambda line: line != header)
    #rdd = rdd.map(lambda tup: RemoveStrangeTupleLen(tup))\
    #         .filter(lambda tup: tup!=None)
             
    rdd = rdd.map(lambda tup: cleaning(tup))
    return(rdd)

INSERT IN THE DB

In [4]:
# we first need to import types (e.g. StructType, StructField, IntegerType, etc.)
from pyspark.sql.types import *

def getlistavion(): 
    QUERY = 'SELECT DISTINCT(TID) FROM global_temp.traffic'
    return spark.sql(QUERY).toPandas()

def main_db(rdd): 
    global traffic_df_explicit, trafficSchema, spark  
    
    traffic_df_explicit_aux = spark.createDataFrame(rdd, trafficSchema)
    traffic_df_explicit = traffic_df_explicit.unionAll(traffic_df_explicit_aux)       
    
    traffic_df_explicit.createOrReplaceGlobalTempView('traffic')
    
    traffic_df_explicit.cache()
    
    #Queries
    #spark.sql("select TID, DST, COUNT(*) from global_temp.traffic WHERE TID != '' GROUP BY TID, DST ORDER BY COUNT(*) DESC").show()
    #spark.sql("select * from global_temp.traffic").show()

#main_db(rdd_traffic_clean)    

In [ ]:
def getPacketsByPlane(tid):
    TID = '\'%' + tid + '%\'' 
    QUERY = 'SELECT * FROM global_temp.traffic \
            WHERE TID LIKE ' + str(TID)
    #print(QUERY)
    return spark.sql(QUERY).toPandas()

def dictRadarsByAvion(TID):
    df_temp = getPacketsByPlane(TID)
    list_radar = list(df_temp.groupby('DST').size().index)
    dictRadar = {}
    for dst in list_radar:
        #dictRadar[dst] = filterByPlaneAndRadar(TID, dst) ##TOO MUCH TIME
        dictRadar[dst] = df_temp[df_temp['DST'] == dst] 
    return dictRadar

In [ ]:
dictRadarsByAvion('TVF168')

In [ ]:
def filterByPlaneAndRadar(tid, dst):
    TID = '\'%' + tid + '%\'' 
    DST = '\'%' + dst + '%\'' 
    QUERY = 'SELECT * FROM global_temp.traffic WHERE TID LIKE ' + str(TID) + ' AND DST LIKE ' +  str(DST) 
    return spark.sql(QUERY).toPandas()

In [ ]:
def countSup20ForTS(tid, dst): 
    TID = '\'%' + tid + '%\'' 
    DST = '\'%' + dst + '%\'' 
    QUERY = 'SELECT * FROM global_temp.traffic WHERE TID LIKE ' + str(TID) + ' AND DST LIKE ' +  str(DST) + ' AND CGS IS NOT NULL' 
    return spark.sql(QUERY).toPandas()

In [ ]:
countSup20ForTS('DSO05LM', '01:00:5e:50:01:42')

In [ ]:
#if len(countSup20ForTS('KLM587', '01:00:5e:50:00:0a')) < 20: 
#    return "TOO MANY VALUES"

In [ ]:
!pip install stumpy

In [ ]:
import numpy as np
import stumpy
import numpy.testing as npt
import time

In [ ]:
def timeseries(df):   
    
    #if len(df) < 20: 
    #    return print("NOT ENOUGH VALUES")
    
    '''
    elif len(df) == 10:    
        #--STREAM--
        #initialisation 
        stream = stumpy.stumpi(T, m)
    else:
        #update t
        stream.update(l)
        #recup les résultats
        ts = pd.Series(df_time_series_radar_avion[var].values, index = df_time_series_radar_avion['TS'])
        ts.plot(color='r')
        
        ts_for = pd.Series(stream.P_, index = df_time_series_radar_avion['TS'] )
        plt.show()
    '''   

    #--BATCH--
    #matrix profile 
    #mp = stumpy.stump(T, m)

    #recup les résultats
    #P_full = mp[:, 0]
    #Ifull = mp[:, 1]
   
    #df_time_series_radar_avion = df.copy()
    #df_time_series_radar_avion['TS_DATE'] = df_time_series_radar_avion['TS'].copy()
    
    #for i in range(len(df_time_series_radar_avion['TS'])):
    #    df_time_series_radar_avion['TS_DATE'][i] = datetime.datetime.fromtimestamp(df_time_series_radar_avion['TS'][i].astype('int64'))
    
    #ts = pd.Series(df_time_series_radar_avion[var].values, index = df_time_series_radar_avion['TS_DATE'])
    #ts.plot(color='r')
    #plt.title('Time Series Plot for CGS Variable over time')
    #plt.show()
    
    #result = seasonal_decompose(ts, model='additive', period=10)
    #result.plot()
    #plt.show()

    #window = 20
    #ma = ts.rolling(window).mean()
    #mstd = ts.rolling(window).std()

    #plt.figure()
    #plt.plot(ts.index, ts, 'r')
    #plt.plot(ma.index, ma, 'b')
    #plt.fill_between(mstd.index, ma - 2 * mstd, ma + 2 * mstd, color='b', alpha=0.2)
    
    ###FBPROPHET###
    
    m = Prophet()
    #m.fit(df_time_series_radar_avion[['TS_DATE','CGS']].rename(columns={'TS_DATE': 'ds', var: "y"}))
    #m.fit(df_time_series_radar_avion[['TS_DATE','CGS']])
    #m.fit(df[['ds','y']])
    
    m.fit(df)
    future = m.make_future_dataframe(periods=2,freq='min')
        #future.tail()
    #print(len(future))
    forecast = m.predict(future)
    m.plot(forecast)
        #m.plot_components(forecast)
        
    '''
    
    ##ARIMA/ARMA Models 
    
    ts_diff = ts - ts.shift()
    diff = ts_diff.dropna()
    plt.figure()
    plt.plot(diff)
    plt.title('First Difference Time Series Plot')
    plt.show()
    mod = ARIMA(ts, order = (1, 1, 1), seasonal_order = (2,0,0,12)) #modèle en dur, à changer 
    results = mod.fit()
    ts.plot()
    plt.plot(results.fittedvalues[20:], color='red')
    plt.show()
    
    predVals = results.predict(len(ts.index), len(ts.index) + 200, typ='levels')
    dict_for = {}
    
    temps = df_time_series_radar_avion['TS'][len(ts.index) - 1]
    for i in range(len(ts.index), len(ts.index) + 200):
        dict_for[datetime.datetime.fromtimestamp(int(temps))] = predVals[i]
        temps = temps + round(df_time_series_radar_avion['TS'][len(ts.index) - 1] - df_time_series_radar_avion['TS'][len(ts.index) - 2])
    
    ts.plot()
    plt.plot(results.fittedvalues[20:], color='red')
    plt.plot(pd.Series(dict_for), color='orange')
    plt.show()
    
    window_for = 20
    ma_for = pd.Series(dict_for).rolling(window).mean()
    mstd_for = pd.Series(dict_for).rolling(window).std()

    plt.figure()
    plt.plot(ts.index, ts, 'b')
    plt.plot(results.fittedvalues[20:], color='red')
    plt.plot(pd.Series(dict_for), color='orange')
    plt.plot(ma_for.index, ma_for, 'b')
    plt.fill_between(mstd_for.index, ma_for - 2 * mstd_for, ma_for + 2 * mstd_for, color='g', alpha=1)
    '''

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("Spark-Dataframe-SQL").getOrCreate()

trafficSchema = StructType ( [StructField("SRC", StringType(), True),
                                 StructField("CAT", LongType(), True),
                                 StructField("TID", StringType(), True),
                                 StructField("TS", DoubleType(), True),
                                 StructField("DST", StringType(), True),
                                 StructField("SAC", DoubleType(), True),
                                 StructField("SIC", DoubleType(), True),
                                 StructField("ToD", DoubleType(), True),
                                 StructField("TN", DoubleType(), True),
                                 StructField("THETA", DoubleType(), True),
                                 StructField("RHO", DoubleType(), True),
                                 StructField("FL", DoubleType(), True),
                                 StructField("CGS", DoubleType(), True),
                                 StructField("CHdg", DoubleType(), True),
                             ] )
    
traffic_df_explicit = spark.createDataFrame(spark.sparkContext.emptyRDD(),trafficSchema)
traffic_df_explicit.createOrReplaceGlobalTempView('traffic')

response = requests.get('http://192.168.37.142:50005/stream/2019-05-04-12:00:00/2019-05-04-16:00:00', stream=True)

#ype(response)
i = 0

list_aux = [] 

for data in response.iter_lines():
    #print(data.decode("UTF-8"))  
    #print(str(data)[1:])
    i = i + 1
    #print(i)
    #print([data.decode("UTF-8").split(",")]) 
    #print(data.decode("UTF-8").split(","))
    
    list_aux.append(data.decode("UTF-8").split(","))
    
    if not(i % 1000):
        #print(len(response.text))
        print(i)
        rdd_traffic = sc.parallelize(list_aux)
        rdd_traffic_clean = main_clean(rdd_traffic)
        
        #print(rdd_traffic_clean.collect())
        main_db(rdd_traffic_clean) 
        list_aux = []     
        
    #time series 
    
    #if not(i % 4000):
        #timeseries(countSup20ForTS('DSO05LM', '01:00:5e:50:01:42'), 'CGS') #how use it in real-time ? b
        
        ###timeseries(df, var)
        ###timeseries(filterByPlaneAndRadar('TOM549','01:00:5e:50:00:26'), 'CGS')
        ####FIN24Q
           
    #rdd_traffic = sc.parallelize([data.decode("UTF-8").split(",")])
    #rdd_traffic_clean = main_clean(rdd_traffic)
    #print(rdd_traffic_clean.collect())
    #main_db(rdd_traffic_clean) 
    
    #data viz 
    
    #if (i == 1000): 
    #    viz(dictRadarsByAvion('EXS29RP'), 'EXS29RP')
    
    #data time series 
    
    if (i==100000): break 

In [ ]:
#In the absence of an inbuit time series library in Spark (inaccessible in spark), the workaround could be Spark Pandas UDF

In [ ]:
#type(traffic_df_explicit)

In [ ]:
spark.sql('select * from global_temp.traffic').show()

In [ ]:
from pyspark.sql.functions import collect_list, struct, to_timestamp

In [ ]:
def transform_data(row):
    """Transform data from pyspark.sql.Row to python dict to be used in rdd."""
    data = row['data']
    
    # Transform [pyspark.sql.Dataframe.Row] -> [dict]
    data_dicts = []
    for d in data:
        data_dicts.append(d.asDict())

    # Convert into pandas dataframe for fbprophet
    data = pd.DataFrame(data_dicts)
    data['ds'] = pd.to_datetime(data['ds'], unit='s')

    return {
        'app' : 'bidule',
        'data': data,
        'metric' : 'bidule',
    }

In [ ]:
def partition_data(d):
    """Split data into training and testing based on timestamp."""
    # Extract data from pd.Dataframe
    data = d['data']

    # Find max timestamp and extract timestamp for start of day
    max_datetime = pd.to_datetime(max(data['ds']))
    start_datetime = max_datetime.replace(hour=00, minute=00, second=00)

    # Extract training data
    train_data = data[data['ds'] < max_datetime]

    # Account for zeros in data while still applying uniform transform
    #train_data['y'] = train_data['y'].apply(lambda x: np.log(x + 1))

    # Assign train/test split
    d['test_data'] = data.loc[(data['ds'] < start_datetime)
                              & (data['ds'] <= max_datetime)]
    d['train_data'] = train_data

    return d

In [ ]:
def create_model(d):
    """Create Prophet model using each input grid parameter set."""
    m = Prophet()
    d['model'] = m

    return d

In [ ]:
def train_model(d):
    """Fit the model using the training data."""
    model = d['model']
    train_data = d['train_data']
    model.fit(train_data)
    d['model'] = model

    return d

In [ ]:
def make_forecast(d):
    """Execute the forecast method on the model to make future predictions."""
    model = d['model']
    future = model.make_future_dataframe(
        periods=0, freq='min')
    
    forecast = model.predict(future)
    d['forecast'] = forecast

    return d

In [ ]:
def reduce_data_scope(d):
    """Return a tuple (app + , + metric_type, {})."""
    return (
        d['app'] + ',' + d['metric'],
        {
            'forecast': pd.concat([d['train_data']['y'],d['forecast']], axis=1),  
        },
    )

In [ ]:
def expand_predictions(d):
    """Flatten rdd into tuple which will be converted into a dataframe.Row.
    Checks each float to see if it is a np datatype, since it could be None.
    If it is an np datatype then it will convert to scalar python datatype
    so that it can be persisted into a database, since most dont know how to
    interpret np python datatypes.
    """
    app_metric, data = d
    app, metric = app_metric.split(',')
    return [
        (
            p['ds'].strftime("%d-%b-%Y (%H:%M:%S)"),
            p['y'],
            p['yhat'],
            p['yhat_lower'],
            p['yhat_upper'],
        ) for i, p in data['forecast'].iterrows()
    ]

In [ ]:
traffic_df_explicit.select(
                    traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
                    traffic_df_explicit['CGS'].alias('y'))\
                   .filter("TID == 'DSO05LM ' and DST == '01:00:5e:50:01:42'")\
                   .agg(collect_list(struct('ds', 'y')).alias('data'))\
                   .rdd.map(lambda r : transform_data(r))\
                       .map(lambda d: partition_data(d))\
                       .filter(lambda d: len(d['train_data']) > 2)\
                       .map(lambda d: create_model(d))\
                       .map(lambda d: train_model(d))\
                       .map(lambda d: make_forecast(d))\
                       .map(lambda d: reduce_data_scope(d))\
                       .flatMap(lambda d: expand_predictions(d))\
                       .take(1)
          
#traffic_df_explicit.cache()

In [ ]:
traffic_for = traffic_df_explicit.select(
                    traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
                    traffic_df_explicit['CGS'].alias('y'))\
                   .filter("TID == 'DSO05LM ' and DST == '01:00:5e:50:01:42'")\
                   .agg(collect_list(struct('ds', 'y')).alias('data'))\
                   .rdd.map(lambda r : transform_data(r))\
                       .map(lambda d: partition_data(d))\
                       .filter(lambda d: len(d['train_data']) > 2)\
                       .map(lambda d: create_model(d))\
                       .map(lambda d: train_model(d))\
                       .map(lambda d: make_forecast(d))\
                       .map(lambda d: reduce_data_scope(d))\
                       .flatMap(lambda d: expand_predictions(d))
          
#type(traffic_for)

In [ ]:
schema_for = StructType([
        StructField("ds", StringType(), True),
        StructField("y", FloatType(), True),
        StructField("yhat", FloatType(), True),
        StructField("yhat_lower", FloatType(), True),
        StructField("yhat_upper", FloatType(), True),
    ])

In [ ]:
df_for = spark.createDataFrame(traffic_for, schema_for)

In [ ]:
df_for.show()

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("Spark-Dataframe-SQL").getOrCreate()

trafficSchema = StructType ( [StructField("SRC", StringType(), True),
                                 StructField("CAT", LongType(), True),
                                 StructField("TID", StringType(), True),
                                 StructField("TS", DoubleType(), True),
                                 StructField("DST", StringType(), True),
                                 StructField("SAC", DoubleType(), True),
                                 StructField("SIC", DoubleType(), True),
                                 StructField("ToD", DoubleType(), True),
                                 StructField("TN", DoubleType(), True),
                                 StructField("THETA", DoubleType(), True),
                                 StructField("RHO", DoubleType(), True),
                                 StructField("FL", DoubleType(), True),
                                 StructField("CGS", DoubleType(), True),
                                 StructField("CHdg", DoubleType(), True),
                             ] )
    
traffic_df_explicit = spark.createDataFrame(spark.sparkContext.emptyRDD(),trafficSchema)
traffic_df_explicit.createOrReplaceGlobalTempView('traffic')

schema_for = StructType([
        StructField("ds", StringType(), True),
        StructField("y", FloatType(), True),
        StructField("yhat", FloatType(), True),
        StructField("yhat_lower", FloatType(), True),
        StructField("yhat_upper", FloatType(), True),
    ])

response = requests.get('http://192.168.37.142:50005/stream/2019-05-04-12:00:00/2019-05-04-16:00:00', stream=True)
#ype(response)
i = 0

list_aux = [] 
cmpt_tram=0

for data in response.iter_lines():
    #print(data.decode("UTF-8"))  
    #print(str(data)[1:])
    i = i + 1
    #print(i)
    #print([data.decode("UTF-8").split(",")]) 
    #print(data.decode("UTF-8").split(","))
    
    ligne = data.decode("UTF-8").split(",")
    list_aux.append(ligne)
    
       #prédire lorsque j'ai 5 nouveaux paquets pour un avion considere et un radar considere 
    #-> prediction 
    #ligne[2] : TID
    #ligne[4] : DST
    
    #Pour l'avion et le radar considere
    if('DSO05LM' in ligne[2] and '01:00:5e:50:01:42' in ligne[4]):
        #compteur pour le nombre de tram
        cmpt_tram += 1       
        #print(compt_tram)
        
        rdd_traffic = sc.parallelize(list_aux)
        rdd_traffic_clean = main_clean(rdd_traffic)
        
        #print(rdd_traffic_clean.collect())
        main_db(rdd_traffic_clean) 
        list_aux = []  
        
        #envoie de la prédiction toutes les 5 trams
        if(cmpt_tram==5):
            #faire la prédiction 
            traffic_for = traffic_df_explicit.select(
                    traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
                    traffic_df_explicit['CGS'].alias('y'))\
                   .filter("TID == 'DSO05LM ' and DST == '01:00:5e:50:01:42'")\
                   .agg(collect_list(struct('ds', 'y')).alias('data'))\
                   .rdd.map(lambda r : transform_data(r))\
                       .map(lambda d: partition_data(d))\
                       .filter(lambda d: len(d['train_data']) > 2)\
                       .map(lambda d: create_model(d))\
                       .map(lambda d: train_model(d))\
                       .map(lambda d: make_forecast(d))\
                       .map(lambda d: reduce_data_scope(d))\
                       .flatMap(lambda d: expand_predictions(d))
        
            traffic_for.cache()
            df_for = spark.createDataFrame(traffic_for, schema_for)
            df_for.show()
            
            #envoie de y et de la prédiction 
            
            #Réinitialisation du compteur
            cmpt_tram=0
        else:
            #Envoie de y
            pass
        
    
    if(not(i%1000)):
        print(i)
        
        
    '''
    list_aux.append(ligne)
    
    if not(i % 1000):
        #print(len(response.text))
        print(i)
        rdd_traffic = sc.parallelize(list_aux)
        rdd_traffic_clean = main_clean(rdd_traffic)
        
        #print(rdd_traffic_clean.collect())
        main_db(rdd_traffic_clean) 
        list_aux = []     
        
    #time series 
    
    if not(i % 4000):
        traffic_for = traffic_df_explicit.select(
                    traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
                    traffic_df_explicit['CGS'].alias('y'))\
                   .filter("TID == 'DSO05LM ' and DST == '01:00:5e:50:01:42'")\
                   .agg(collect_list(struct('ds', 'y')).alias('data'))\
                   .rdd.map(lambda r : transform_data(r))\
                       .map(lambda d: partition_data(d))\
                       .filter(lambda d: len(d['train_data']) > 2)\
                       .map(lambda d: create_model(d))\
                       .map(lambda d: train_model(d))\
                       .map(lambda d: make_forecast(d))\
                       .map(lambda d: reduce_data_scope(d))\
                       .flatMap(lambda d: expand_predictions(d))
        
        traffic_for.cache()
        
        df_for = spark.createDataFrame(traffic_for, schema_for)
        
        df_for.show()
    '''
        
    if (i==100000): break 

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("Spark-Dataframe-SQL").getOrCreate()

trafficSchema = StructType ( [StructField("SRC", StringType(), True),
                                 StructField("CAT", LongType(), True),
                                 StructField("TID", StringType(), True),
                                 StructField("TS", DoubleType(), True),
                                 StructField("DST", StringType(), True),
                                 StructField("SAC", DoubleType(), True),
                                 StructField("SIC", DoubleType(), True),
                                 StructField("ToD", DoubleType(), True),
                                 StructField("TN", DoubleType(), True),
                                 StructField("THETA", DoubleType(), True),
                                 StructField("RHO", DoubleType(), True),
                                 StructField("FL", DoubleType(), True),
                                 StructField("CGS", DoubleType(), True),
                                 StructField("CHdg", DoubleType(), True),
                             ] )
    
traffic_df_explicit = spark.createDataFrame(spark.sparkContext.emptyRDD(),trafficSchema)
traffic_df_explicit.createOrReplaceGlobalTempView('traffic')

schema_for = StructType([
        StructField("ds", StringType(), True),
        StructField("y", FloatType(), True),
        StructField("yhat", FloatType(), True),
        StructField("yhat_lower", FloatType(), True),
        StructField("yhat_upper", FloatType(), True),
    ])

response = requests.get('http://192.168.37.142:50005/stream/2019-05-04-12:00:00/2019-05-04-16:00:00', stream=True)
#ype(response)
i = 0

list_aux = [] 
cmpt_tram=0

for data in response.iter_lines():
    #print(data.decode("UTF-8"))  
    #print(str(data)[1:])
    i = i + 1
    #print(i)
    #print([data.decode("UTF-8").split(",")]) 
    #print(data.decode("UTF-8").split(","))
    
    ligne = data.decode("UTF-8").split(",")
    
       #prédire lorsque j'ai 5 nouveaux paquets pour un avion considere et un radar considere 
    #-> prediction 
    #ligne[2] : TID
    #ligne[4] : DST
    
    #Pour l'avion et le radar considere
    if('TOM97V' in ligne[2] and '01:00:5e:50:00:62' in ligne[4]):
        #compteur pour le nombre de tram
        cmpt_tram += 1
        
        #clean
        
        
        if(cmpt_tram==5):
            #faire la prédiction 
            
            #et de la prédiction 
            
            #Réinitialisation du compteur
            cmpt_tram=0
        else:
            #Envoie de y
        
    
    if(not i%500):
        print(cmpt_tram)
        
        
    '''
    list_aux.append(ligne)
    
    if not(i % 1000):
        #print(len(response.text))
        print(i)
        rdd_traffic = sc.parallelize(list_aux)
        rdd_traffic_clean = main_clean(rdd_traffic)
        
        #print(rdd_traffic_clean.collect())
        main_db(rdd_traffic_clean) 
        list_aux = []     
        
    #time series 
    
    if not(i % 4000):
        traffic_for = traffic_df_explicit.select(
                    traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
                    traffic_df_explicit['CGS'].alias('y'))\
                   .filter("TID == 'DSO05LM ' and DST == '01:00:5e:50:01:42'")\
                   .agg(collect_list(struct('ds', 'y')).alias('data'))\
                   .rdd.map(lambda r : transform_data(r))\
                       .map(lambda d: partition_data(d))\
                       .filter(lambda d: len(d['train_data']) > 2)\
                       .map(lambda d: create_model(d))\
                       .map(lambda d: train_model(d))\
                       .map(lambda d: make_forecast(d))\
                       .map(lambda d: reduce_data_scope(d))\
                       .flatMap(lambda d: expand_predictions(d))
        
        traffic_for.cache()
        
        df_for = spark.createDataFrame(traffic_for, schema_for)
        
        df_for.show()
    '''
        
    if (i==100000): break 

## Multiple planes and multiple radars

In [ ]:
spark.sql("select DST, TID, count(*) from global_temp.traffic group by DST, TID order by DST, count(*) DESC").show()

In [5]:
from pyspark.sql.functions import collect_list, struct, to_timestamp

In [6]:
def transform_data_m(row):
    """Transform data from pyspark.sql.Row to python dict to be used in rdd."""
    data = row['data']
    tid = row['TID']
    dst = row['DST']
    
    # Transform [pyspark.sql.Dataframe.Row] -> [dict]
    data_dicts = []
    for d in data:
        data_dicts.append(d.asDict())

    # Convert into pandas dataframe for fbprophet
    data = pd.DataFrame(data_dicts)
    data['ds'] = pd.to_datetime(data['ds'], unit='s')

    return {
        'tid' : tid,
        'dst' : dst,
        'data': data,
    }

In [7]:
def partition_data_m(d):
    """Split data into training and testing based on timestamp."""
    # Extract data from pd.Dataframe
    data = d['data']

    # Find max timestamp and extract timestamp for start of day
    max_datetime = pd.to_datetime(max(data['ds']))
    start_datetime = max_datetime.replace(hour=00, minute=00, second=00)

    # Extract training data
    train_data = data[data['ds'] <= max_datetime]

    # Account for zeros in data while still applying uniform transform
    #train_data['y'] = train_data['y'].apply(lambda x: np.log(x + 1))

    # Assign train/test split
    d['test_data'] = data.loc[(data['ds'] < start_datetime)
                              & (data['ds'] <= max_datetime)]
    d['train_data'] = train_data

    return d

In [8]:
def create_model_m(d):
    """Create Prophet model using each input grid parameter set."""
    m = Prophet()
    d['model'] = m

    return d

In [9]:
def train_model_m(d):
    """Fit the model using the training data."""
    model = d['model']
    train_data = d['train_data']
    model.fit(train_data)
    d['model'] = model

    return d

In [19]:
def make_forecast_m(d):
    """Execute the forecast method on the model to make future predictions."""
    model = d['model']
    future = model.make_future_dataframe(
        periods=2, freq='min')
    
    forecast = model.predict(future)
    d['forecast'] = forecast

    return d

In [11]:
def reduce_data_scope_m(d):
    """Return a tuple (app + , + metric_type, {})."""
    return (
        d['tid'] + ',' + d['dst'],
        {
            'forecast': pd.concat([d['train_data']['y'],d['forecast']], axis=1),  
        },
    )

In [12]:
def expand_predictions_m(d):
    tid_dst, data = d
    tid, dst = tid_dst.split(',')
    return [
        (
            tid, 
            dst,
            p['ds'].strftime("%d-%b-%Y (%H:%M:%S)"),
            p['y'],
            p['yhat'],
            p['yhat_lower'],
            p['yhat_upper'],
        ) for i, p in data['forecast'].iterrows()
    ]

In [ ]:
#spark.sql("select DST, TID, count(*) from global_temp.traffic group by DST, TID order by DST, count(*) DESC").show()

In [ ]:
'''
traffic_df_explicit.select(
    traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
    traffic_df_explicit['CGS'].alias('y'))\
                              .filter("TID like '%DSO05LM%' and DST like '%01:00:5e:50:01:42%'")\
                              .agg(collect_list(struct('ds', 'y')).alias('data'))\
                              .rdd.map(lambda r : transform_data(r))\
                                  .map(lambda d: partition_data(d))\
                                  .filter(lambda d: len(d['train_data']) > 2)\
                                  .map(lambda d: create_model(d))\
                                  .map(lambda d: train_model(d))\
                                  .map(lambda d: make_forecast(d))\
                                  .map(lambda d: reduce_data_scope(d))\
                                  .flatMap(lambda d: expand_predictions(d))\
                                  .take(1)
'''

In [ ]:
traffic_for_m = traffic_df_explicit.select(
                     traffic_df_explicit['TID'],
                     traffic_df_explicit['DST'],                    
                     traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
                     traffic_df_explicit['CGS'].alias('y'))\
                   .filter("(TID like '%AFR15MD%' or TID like '%EWG7XB%') and DST like '%01:00:5e:50:00:06%'")\
                   .groupBy('TID', 'DST')\
                   .agg(collect_list(struct('ds', 'y')).alias('data'))\
                   .rdd.map(lambda r: transform_data_m(r))\
                       .map(lambda d: partition_data_m(d))\
                       .filter(lambda d: len(d['train_data']) > 2)\
                       .map(lambda d: create_model_m(d))\
                       .map(lambda d: train_model_m(d))\
                       .map(lambda d: make_forecast_m(d))\
                       .map(lambda d: reduce_data_scope_m(d))\
                       .flatMap(lambda d: expand_predictions_m(d))\
                   #.rdd.map(lambda r : transform_data_m(r))\
                   #    .map(lambda d: partition_data_m(d))\
                   #    .filter(lambda d: len(d['train_data']) > 2)\
                   #    .map(lambda d: create_model_m(d))\
                      

#01:00:5e:50:00:06                    
#01:00:5e:50:00:06|EWG7XB  |      11|
#|01:00:5e:50:00:06|HVN18   |      11|
#|01:00:5e:50:00:06|EZY47UA

In [ ]:
schema_for_m = StructType([
        StructField("tid", StringType(), True),
        StructField("dst", StringType(), True),
        StructField("ds", StringType(), True),
        StructField("y", FloatType(), True),
        StructField("yhat", FloatType(), True),
        StructField("yhat_lower", FloatType(), True),
        StructField("yhat_upper", FloatType(), True),
    ])

In [ ]:
df_for = spark.createDataFrame(traffic_for_m, schema_for_m)

In [ ]:
df_for.show()

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("Spark-Dataframe-SQL").getOrCreate()

trafficSchema = StructType ( [StructField("SRC", StringType(), True),
                                 StructField("CAT", LongType(), True),
                                 StructField("TID", StringType(), True),
                                 StructField("TS", DoubleType(), True),
                                 StructField("DST", StringType(), True),
                                 StructField("SAC", DoubleType(), True),
                                 StructField("SIC", DoubleType(), True),
                                 StructField("ToD", DoubleType(), True),
                                 StructField("TN", DoubleType(), True),
                                 StructField("THETA", DoubleType(), True),
                                 StructField("RHO", DoubleType(), True),
                                 StructField("FL", DoubleType(), True),
                                 StructField("CGS", DoubleType(), True),
                                 StructField("CHdg", DoubleType(), True),
                             ] )
    
traffic_df_explicit = spark.createDataFrame(spark.sparkContext.emptyRDD(),trafficSchema)
traffic_df_explicit.createOrReplaceGlobalTempView('traffic')

schema_for_m = StructType([
        StructField("tid", StringType(), True),
        StructField("dst", StringType(), True),
        StructField("ds", StringType(), True),
        StructField("y", FloatType(), True),
        StructField("yhat", FloatType(), True),
        StructField("yhat_lower", FloatType(), True),
        StructField("yhat_upper", FloatType(), True),
    ])

response = requests.get('http://192.168.37.142:50005/stream/2019-05-04-12:00:00/2019-05-04-16:00:00', stream=True)
#ype(response)
i = 0

list_aux = [] 

for data in response.iter_lines():
    #print(data.decode("UTF-8"))  
    #print(str(data)[1:])
    i = i + 1
    #print(i)
    #print([data.decode("UTF-8").split(",")]) 
    #print(data.decode("UTF-8").split(","))
    
    list_aux.append(data.decode("UTF-8").split(","))
    
    if not(i % 1000):
        #print(len(response.text))
        print(i)
        rdd_traffic = sc.parallelize(list_aux)
        rdd_traffic_clean = main_clean(rdd_traffic)
        
        #print(rdd_traffic_clean.collect())
        main_db(rdd_traffic_clean) 
        list_aux = []     
        
    #time series 
    
    if not(i % 4000):
        traffic_for_m = traffic_df_explicit.select(
                     traffic_df_explicit['TID'],
                     traffic_df_explicit['DST'],                    
                     traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
                     traffic_df_explicit['CGS'].alias('y'))\
                   .filter("TID like '%AFR15MD%' and DST like '%01:00:5e:50:00:06%'")\
                   .groupBy('TID', 'DST')\
                   .agg(collect_list(struct('ds', 'y')).alias('data'))\
                   .rdd.map(lambda r: transform_data_m(r))\
                       .map(lambda d: partition_data_m(d))\
                       .filter(lambda d: len(d['train_data']) > 2)\
                       .map(lambda d: create_model_m(d))\
                       .map(lambda d: train_model_m(d))\
                       .map(lambda d: make_forecast_m(d))\
                       .map(lambda d: reduce_data_scope_m(d))\
                       .flatMap(lambda d: expand_predictions_m(d))\
        
        traffic_for_m.cache()
        
        df_for_m = spark.createDataFrame(traffic_for_m, schema_for_m)
        
        df_for_m.show()
        
    if (i==100000): break 

In [36]:
spark = pyspark.sql.SparkSession.builder.appName("Spark-Dataframe-SQL").getOrCreate()

trafficSchema = StructType ( [StructField("SRC", StringType(), True),
                                 StructField("CAT", LongType(), True),
                                 StructField("TID", StringType(), True),
                                 StructField("TS", DoubleType(), True),
                                 StructField("DST", StringType(), True),
                                 StructField("SAC", DoubleType(), True),
                                 StructField("SIC", DoubleType(), True),
                                 StructField("ToD", DoubleType(), True),
                                 StructField("TN", DoubleType(), True),
                                 StructField("THETA", DoubleType(), True),
                                 StructField("RHO", DoubleType(), True),
                                 StructField("FL", DoubleType(), True),
                                 StructField("CGS", DoubleType(), True),
                                 StructField("CHdg", DoubleType(), True),
                             ] )
    
traffic_df_explicit = spark.createDataFrame(spark.sparkContext.emptyRDD(),trafficSchema)
traffic_df_explicit.createOrReplaceGlobalTempView('traffic')

schema_for_m = StructType([
        StructField("tid", StringType(), True),
        StructField("dst", StringType(), True),
        StructField("ds", StringType(), True),
        StructField("y", FloatType(), True),
        StructField("yhat", FloatType(), True),
        StructField("yhat_lower", FloatType(), True),
        StructField("yhat_upper", FloatType(), True),
    ])
    
list_aux = [] 
cmpt_tram = 0        

response = requests.get('http://192.168.37.142:50005/stream/2019-05-04-12:00:00/2019-05-04-16:00:00', stream=True)
#type(response)

i = 0
for data in response.iter_lines():
    #print(data.decode("UTF-8"))  
    #print(str(data)[1:])
    i = i + 1
    #print(i)
    #print([data.decode("UTF-8").split(",")]) 
    #print(data.decode("UTF-8").split(","))
    
    ligne = data.decode("UTF-8").split(",")
    list_aux.append(ligne)
    
      
    #prédire lorsque j'ai 5 nouveaux paquets pour un avion considere et un radar considere 
    #-> prediction 
    #ligne[2] : TID
    #ligne[4] : DST
    
    #Pour l'avion et le radar considere
    if('DSO05LM' in ligne[2] and '01:00:5e:50:01:42' in ligne[4]):
        #compteur pour le nombre de tram   
        print(ligne)
        
        #print(compt_tram)
        
        rdd_traffic = sc.parallelize(list_aux)
        rdd_traffic_clean = main_clean(rdd_traffic)
        
        #print(rdd_traffic_clean.collect())
        main_db(rdd_traffic_clean) 
        
        #faire un show pour un envoi en temps réel à la base de données sql
        
        cmpt_tram += 1 
        list_aux = []     
        
    #time series 
       
        #envoie de la prédiction toutes les 5 trams
        if(cmpt_tram==5):
            #faire la prédiction 
            traffic_for_m = traffic_df_explicit.select(
                     traffic_df_explicit['TID'],
                     traffic_df_explicit['DST'],                    
                     traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
                     traffic_df_explicit['CGS'].alias('y'))\
                   .filter("TID like '%DSO05LM%' and DST like '%01:00:5e:50:01:42%'")\
                   .groupBy('TID', 'DST')\
                   .agg(collect_list(struct('ds', 'y')).alias('data'))\
                   .rdd.map(lambda r: transform_data_m(r))\
                       .map(lambda d: partition_data_m(d))\
                       .filter(lambda d: len(d['train_data']) > 2)\
                       .map(lambda d: create_model_m(d))\
                       .map(lambda d: train_model_m(d))\
                       .map(lambda d: make_forecast_m(d))\
                       .map(lambda d: reduce_data_scope_m(d))\
                       .flatMap(lambda d: expand_predictions_m(d))\
        
            traffic_for_m.cache()
        
            df_for_m = spark.createDataFrame(traffic_for_m, schema_for_m)
        
            df_for_m.show()
            
            #envoie de y et de la prédiction 
            
            #Réinitialisation du compteur
            cmpt_tram=0
        else:
            #Envoie de y
            
            #clean et envoi de la ligne à la volée
        
            tid = ligne[2]
            dst = ligne[4]
            ds = pd.to_datetime(float(round(float(ligne[3]))), unit='s').strftime("%d-%b-%Y (%H:%M:%S)")
            y = float(ligne[13])
            yhat = 'NaN'
            yhat_lower = 'NaN'
            yhat_upper = 'NaN'
        
            d = {'tid': [tid], 'dst': [dst], 'ds': [ds], 'y': [y], 'yhat': [yhat], 
             'yhat_lower': [yhat_lower], 'yhat_upper': [yhat_upper]}
        
            #spark.createDataFrame(traffic_for_m, schema_for_m).show()
        
            print(pd.DataFrame(data=d))
    
    if(not(i%1000)):
        print(i)
        
    if (i==100000): break 

['00:23:9c:c1:94:5f', '48', 'DSO05LM ', '1556976195.58', '01:00:5e:50:01:42', '8', '80', '48194.625', '1253', '349.738769531', '10.65234375', '79.5', '300.366210938', '212.135009766']
        tid                dst                      ds          y yhat  \
0  DSO05LM   01:00:5e:50:01:42  04-May-2019 (13:23:16)  212.13501  NaN   

  yhat_lower yhat_upper  
0        NaN        NaN  
1000
2000
3000
4000
['00:23:9c:c1:94:5f', '48', 'DSO05LM ', '1556976199.49', '01:00:5e:50:01:42', '8', '80', '48198.5625', '1253', '348.711547852', '10.37890625', '79.5', '303.662109375', '195.485229492']
        tid                dst                      ds           y yhat  \
0  DSO05LM   01:00:5e:50:01:42  04-May-2019 (13:23:19)  195.485229  NaN   

  yhat_lower yhat_upper  
0        NaN        NaN  
5000
6000
7000
8000
9000
['00:23:9c:c1:94:5f', '48', 'DSO05LM ', '1556976203.44', '01:00:5e:50:01:42', '8', '80', '48202.515625', '1253', '347.750244141', '10.09375', '79.75', '305.419921875', '197.775878906

KeyboardInterrupt: 

######## 

In [26]:
traffic_df_explicit.select(
                     traffic_df_explicit['TID'],
                     traffic_df_explicit['DST'],                    
                     traffic_df_explicit['TS'].cast(IntegerType()).alias('ds'), 
                     traffic_df_explicit['CGS'].alias('y'))\
                   .filter("TID like '%DSO05LM%' and DST like '%01:00:5e:50:01:42%'")\
                   .groupBy('TID', 'DST')\
                   .agg(collect_list(struct('ds', 'y')).alias('data'))\
                   .rdd.map(lambda r: transform_data_m(r))\
                       .map(lambda d: partition_data_m(d))\
                       .map(lambda d: create_model_m(d))\
                       .map(lambda d: train_model_m(d))\
                       .map(lambda d: make_forecast_m(d))\
                       .map(lambda d: reduce_data_scope_m(d))\
                       .flatMap(lambda d: expand_predictions_m(d))\
                       .collect()

[('DSO05LM ',
  '01:00:5e:50:01:42',
  '04-May-2019 (13:23:16)',
  300.366210938,
  301.9432051753301,
  299.4996831172537,
  304.3578307972929),
 ('DSO05LM ',
  '01:00:5e:50:01:42',
  '04-May-2019 (13:23:19)',
  303.662109375,
  302.23578385392193,
  299.8368285393827,
  304.75964884276175),
 ('DSO05LM ',
  '01:00:5e:50:01:42',
  '04-May-2019 (13:23:23)',
  305.419921875,
  302.62588876084345,
  300.0737033971831,
  305.37769154630035),
 ('DSO05LM ',
  '01:00:5e:50:01:42',
  '04-May-2019 (13:23:27)',
  301.245117188,
  303.015993019659,
  300.5175977625569,
  305.5127219226727),
 ('DSO05LM ',
  '01:00:5e:50:01:42',
  '04-May-2019 (13:23:31)',
  300.5859375,
  303.4060972731281,
  300.75228698414617,
  305.75120756802954),
 ('DSO05LM ',
  '01:00:5e:50:01:42',
  '04-May-2019 (13:23:35)',
  303.22265625,
  303.814834587964,
  301.23142192986376,
  306.15416264321567),
 ('DSO05LM ',
  '01:00:5e:50:01:42',
  '04-May-2019 (13:23:39)',
  306.298828125,
  304.223571981109,
  301.8971216036121

Discretized Streams

In [ ]:
#If the batch duration is 2 seconds, then the data will be collected every 2 seconds and stored in an RDD

DataFrame and SQL Operations

In [ ]:
'''
import mysql.connector as mconn
from datetime import datetime
from time import sleep

def create_database(database_name="test"):
    '''
    #create a database using: CREATE DATABASE database_name;
    #select a database using: USE database_name '''
    conn = mconn.connect(host="localhost", port=3306, user="root", password="secret")
    cur = conn.cursor()
    cur.execute("CREATE DATABASE %s;"%(database_name))
    conn.commit()
    conn.close()

def show_available_databases():
    conn = mconn.connect(host="localhost", port=3306, user="root", password="secret")
    cur = conn.cursor()
    cur.execute("SHOW DATABASES;")
    databases = cur.fetchall()
    for database in databases:
        print(database)
    conn.close()

def create_table(table_name="tabl1", database_name="test"):
    conn = mconn.connect(host="localhost", port=3306, user="root", password="secret", database=database_name)
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS %s (x INT, y FLOAT, y_low FLOAT, y_upp FLOAT);"%(table_name))
    conn.commit()
    conn.close()

def drop_table(table_name="tabl1", database_name="test"):
    conn = mconn.connect(host="localhost", port=3306, user="root", password="secret", database=database_name)
    cur = conn.cursor()
    cur.execute("DROP TABLE %s;"%(table_name))
    conn.commit()
    conn.close()

def draft_populate(table_name="tabl1", database_name="test"):
    conn = mconn.connect(host="localhost", port=3306, user="root", password="secret", database=database_name)
    cur = conn.cursor()
    with open("pourKaderEtCharles.csv", "r") as f:
        next(f)
        for line_ in f:
            line = line_.replace("\n", "")
            _,y,ds,yhat_lower,yhat_upper,TID,DST = line.split(",")
            t = datetime.strptime(ds,"%Y-%m-%d %H:%M:%S")
            # print("==%s=="%(y))
            cur.execute("INSERT INTO %s VALUES (%s, %s, %s, %s);"%(table_name,(t-datetime(1970,1,1)).total_seconds(), float(y), float(yhat_lower), float(yhat_upper)))
            conn.commit()
            sleep(2)
        f.close()
    conn.close()

def query_from_table(table_name="tabl1", database_name="test"):
    conn = mconn.connect(host="localhost", port=3306, user="root", password="secret", database=database_name)
    cur = conn.cursor()
    cur.execute("SELECT * FROM %s;"%(table_name))
    rows = cur.fetchall()
    conn.close()
    for row in rows:
        print(row)

# create_database()
# show_available_databases()
# create_table()
draft_populate()
# query_from_table()
# drop_table()
'''

In [ ]:
#A chaque insert dans la base, une mise à jour des plots